In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
import datetime
from datetime import date

In [10]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

driver.get("https://www.billboard.com/charts/billboard-global-200/")

driver.find_element(By.XPATH,"//*[@id='onetrust-reject-all-handler']").click()
# enlever la box privacy policy
# driver.find_element(By.XPATH,"/html/body/div[5]/div[1]/span").click()

iterateur100 = list(range(1,110,11))
iterateur100.remove(1)
iterateur200 = list(range(1,220,11))
iterateur200.remove(1)

XpathTitle = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[1]/h3"
XpathArtist = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[1]/span"
XpathRank = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[1]/span "
XpathLastWeek = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[4]/span"
XpathPeakPosition = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[5]/span"
XpathWeeksOnChart = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[6]/span"

allXpath = [XpathTitle,XpathArtist,XpathRank,XpathLastWeek,XpathPeakPosition,XpathWeeksOnChart]

title,artist,rank,last_week,peak_pos,weeks_on_chart = [],[],[],[],[],[]

every_date,list_date = [],[]

allList = [title,artist,rank,last_week,peak_pos,weeks_on_chart]

# nb_max 111 ou 221
def scraper(list,xpath,iterateur,nb_max):
    for i in range(2,nb_max):
        if i not in iterateur : 
            WebElement = driver.find_element(By.XPATH,xpath%(i))
            list.append(WebElement.text)

    return list

def get_dates_of_year_by_week(year, nb_mois):
    starting_week = 52 - 4 * nb_mois
    current_week = starting_week
    first_day_of_year = datetime.datetime(year, 1, 1)
    week_start = first_day_of_year + datetime.timedelta(days=(current_week - 1) * 7)
    
    while week_start.year == year:
        every_date.append(week_start)
        current_week += 1
        week_start = first_day_of_year + datetime.timedelta(days=(current_week - 1) * 7)
    
    return every_date

def first_day_of_each_week_for_year(year):
    first_day_of_year = datetime.datetime(year, 1, 1)
    one_week = datetime.timedelta(weeks=1)
    current_week = first_day_of_year
    every_date.append(first_day_of_year)

    while current_week.year == year:
        current_week += one_week
        every_date.append(current_week)

    return every_date
        
today = date.today()

get_dates_of_year_by_week(2022,3)
first_day_of_each_week_for_year(2023)

for elt_date in every_date:
    elt_date = elt_date.strftime("%Y-%m-%d")
    today = date.today() - datetime.timedelta(weeks=1)
    if elt_date >= today.strftime("%Y-%m-%d"):
        break
    else : 
        # get on the correct billboard page 
        driver.get(f"https://www.billboard.com/charts/billboard-global-200/{elt_date}")
        # create a list with the date for all entries (200 per date) 
        [list_date.append(elt_date) for i in range(200)]
        # scrape all the data
        i = [scraper(i,j,iterateur200,221) for i, j in zip(allList, allXpath)]

df = pd.DataFrame(list(zip(title,artist,rank,last_week,peak_pos,weeks_on_chart,list_date)),columns=['Title','Artist','Rank','Last Week','Peak Positon','Weeks on charts','Date'])

driver.close() 

df

C:\Users\camil\AppData\Local\Temp\ipykernel_16884\2076613950.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


,Title,Artist,Rank,Last Week,Peak Positon,Weeks on charts,Date
0,All I Want For Christmas Is You,Mariah Carey,1,1,1,28,2023-01-01
1,Last Christmas,Wham!,2,2,2,24,2023-01-01
2,Rockin' Around The Christmas Tree,Brenda Lee,3,3,2,21,2023-01-01
3,Jingle Bell Rock,Bobby Helms,4,4,4,21,2023-01-01
4,Feliz Navidad,Jose Feliciano,5,10,5,19,2023-01-01
...,...,...,...,...,...,...,...
1395,In The End,Linkin Park,196,193,163,12,2023-02-12
1396,Ferrari,James Hype & Miggy Dela Rosa,197,174,25,31,2023-02-12
1397,Just The Way You Are,Bruno Mars,198,-,162,12,2023-02-12
1398,Call Out My Name,The Weeknd,199,181,97,31,2023-02-12


In [20]:
df.to_csv('dataframe.csv',sep=';',index=False)

In [1]:
# runtime scraping environ 12 min 30 sec sans headless browser
# 6min 30 pour 1400 rows en headless
# 10 min pour 2000 rows

In [46]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
# driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://www.billboard.com/charts/billboard-global-200/")

driver.find_element(By.XPATH,"//*[@id='onetrust-reject-all-handler']").click()

title,artist,rank,last_week,peak_pos,weeks_on_chart = [],[],[],[],[],[]

every_date,list_date = [],[]

def scraper ():

    all = driver.find_elements(By.CLASS_NAME,"c-label")
    print(len(all))

    all_title = driver.find_elements(By.XPATH,"//*[@id='title-of-a-story']")
    title = [elt.text  for elt in all_title]

    list_all = [elt.text  for elt in all]
    list_all = list_all[3:1636]

    new_list = [x for x in list_all if x != '' and  x!='NEW' and x!='RE- ENTRY']

    artist = new_list[1::5]
    rank = new_list[0::5]
    last_week = new_list[2::5]
    weeks_on_chart = new_list[3::5]
    peak_pos = new_list[4::5]

    return title,artist,rank,last_week,peak_pos,weeks_on_chart


def get_dates_of_year_by_week(year, nb_mois):
    starting_week = 52 - 4 * nb_mois
    current_week = starting_week
    first_day_of_year = datetime.datetime(year, 1, 1)
    week_start = first_day_of_year + datetime.timedelta(days=(current_week - 1) * 7)
    
    while week_start.year == year:
        every_date.append(week_start)
        current_week += 1
        week_start = first_day_of_year + datetime.timedelta(days=(current_week - 1) * 7)
    
    return every_date

def first_day_of_each_week_for_year(year):
    first_day_of_year = datetime.datetime(year, 1, 1)
    one_week = datetime.timedelta(weeks=1)
    current_week = first_day_of_year
    every_date.append(first_day_of_year)

    while current_week.year == year:
        current_week += one_week
        every_date.append(current_week)

    return every_date
        
today = date.today()

get_dates_of_year_by_week(2022,3)
first_day_of_each_week_for_year(2023)

for elt_date in every_date:
    elt_date = elt_date.strftime("%Y-%m-%d")
    today = date.today() - datetime.timedelta(weeks=1)

    if elt_date >= today.strftime("%Y-%m-%d"):
        break
    else : 
        # get on the correct billboard page 
        driver.get(f"https://www.billboard.com/charts/billboard-global-200/{elt_date}")
        # create a list with the date for all entries (200 per date) 
        [list_date.append(elt_date) for i in range(200)]
        # scrape all the data
        i = scraper()

df = pd.DataFrame(list(zip(title,artist,rank,last_week,peak_pos,weeks_on_chart,list_date)),columns=['Title','Artist','Rank','Last Week','Peak Positon','Weeks on charts','Date'])

driver.close() 

df

C:\Users\camil\AppData\Local\Temp\ipykernel_16700\3796234520.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


1635
1637
1632
1634


KeyboardInterrupt: 